# Exploratory Analysis

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv ../../.env

import os, sys
import collections

sys.path.append(os.path.abspath('../src'))

from file_handler import download_data

In [ ]:
download_data("../data")

Data already downloaded.


From a glance, we have optical - thermal videos and images. There is a test-dev set presumably for debugging with a much larger trainval set

In [13]:
def get_number_of_files_in_folder(folder):
    counter = collections.Counter()
    for root, dirs, files in os.walk(f"../data/Images/{folder}/Optical"):
        for dir in dirs:
            counter[dir] += len(os.listdir(os.path.join(root, dir)))
    
    print(f'Number of folders in {folder}: {len(counter)}')
    
    # Total number of files
    total = sum(counter.values())
        
    print(f'Total number of files in {folder}: {total}')
    
    return counter


dev_counter = get_number_of_files_in_folder('test-dev')
train_counter = get_number_of_files_in_folder('trainval')

Number of folders in test-dev: 6
Total number of files in test-dev: 1675
Number of folders in trainval: 30
Total number of files in trainval: 8714


## Evaluation Testing

In [2]:
from vos_benchmark.benchmark import benchmark

In [13]:
gt_dir = "../data/optic_thermal/Images/trainval/val/DAVIS_Masks"
pred_dir = "../results/2025-01-25_17-46-41/results/Annotations"

results = benchmark([gt_dir], [pred_dir])

J, JF, F = results[:3]
print(f"J: {J}, JF: {JF}, F: {F}")

We are *SKIPPING* the evaluation of the first and the last frame (standard for semi-supervised video object segmentation).
In dataset ../data/optic_thermal/Images/trainval/val/DAVIS_Masks, we are evaluating on 3 videos: ['Mavic1_DJI_0872', 'Mavic1_DJI_0876', 'Mavic1_DJI_0882']


100%|██████████| 3/3 [00:13<00:00,  4.48s/it]

sequence        obj   J&F     J     F
Global score         11.1  6.3  15.9
Mavic1_DJI_0872 001  11.6   7.5  15.8
Mavic1_DJI_0876 001  18.1  10.7  25.5
Mavic1_DJI_0882 001   3.6   0.7   6.5

Summary:
Global score: J&F: 11.1 J: 6.3 F: 15.9
Time taken: 13.48s
J: [11.115216914350716], JF: [6.2982486238266695], F: [15.932185204874763]
